In [1]:
import json
import os
import numpy as np

In [2]:
def import_metrics_json(models_path, run_num):
    """
    Looks in {models_path}/{run_num}/metrics.json and returns the contents as a
    Python dictionary. Returns None if the path does not exist.
    """
    path = os.path.join(models_path, str(run_num), "metrics.json")
    if not os.path.exists(path):
        return None
    with open(path, "r") as f:
        return json.load(f)

In [3]:
def get_best_metric(models_path, metric_name, reduce_func, compare_func):
    """
    Given the path to a set of runs, determines the run with the best metric value,
    for the given `metric_name`. For each run, the function `reduce_func` must take
    the array of all values for that metric and return a (scalar) value to use for
    comparison. The best metric value is determined by `metric_compare_func`, which
    must take in two arguments, and return whether or not the _first_ one is better.
    Returns the number of the run, the value associated with that run, and a dict of
    all the values used for comparison.
    """
    # Get the metrics, ignoring empty or nonexistent metrics.json files
    metrics = {run_num : import_metrics_json(models_path, run_num) for run_num in os.listdir(models_path)}
    metrics = {key : val for key, val in metrics.items() if val}  # Remove empties
    
    # Get the best value
    best_run, best_val, all_vals = None, None, {}
    for run_num in metrics.keys():
        try:
            val = reduce_func(metrics[run_num][metric_name]["values"])
            all_vals[run_num] = val
            if best_val is None or compare_func(val, best_val):
                best_val, best_run = val, run_num
        except Exception:
            print("Warning: Was not able to compute values for run %s" % run_num)
            continue
    return best_run, best_val, all_vals

In [4]:
def get_best_metric_at_best_epoch(models_path, metric_name, reduce_func, compare_func):
    """
    Given the path to a set of runs, determines the run with the best metric value,
    for the given `metric_name`. For each run, the function `reduce_func` must take
    the array of all values for that metric and return a (scalar) value FOR EACH
    SUBARRAY/VALUE in the value array to use for comparison. The best metric value
    is determined by `metric_compare_func`, which must take in two arguments, and
    return whether or not the _first_ one is better.
    Returns the number of the run, the (one-indexed) number of the epch, the value
    associated with that run and epoch, and a dict of all the values used for
    comparison (mapping pair of run number and epoch number to value).
    """
    # Get the metrics, ignoring empty or nonexistent metrics.json files
    metrics = {run_num : import_metrics_json(models_path, run_num) for run_num in os.listdir(models_path)}
    metrics = {key : val for key, val in metrics.items() if val}  # Remove empties
    
    # Get the best value
    best_run, best_epoch, best_val, all_vals = None, None, None, {}
    for run_num in metrics.keys():
        try:
            # Find the best epoch within that run
            best_epoch_in_run, best_val_in_run = None, None
            for i, subarr in enumerate(metrics[run_num][metric_name]["values"]):
                val = reduce_func(subarr)
                if best_val_in_run is None or compare_func(val, best_val_in_run):
                    best_epoch_in_run, best_val_in_run = i + 1, val
            all_vals[(run_num, best_epoch_in_run)] = best_val_in_run
            
            # If the best value in the best epoch of the run is best so far, update
            if best_val is None or compare_func(best_val_in_run, best_val):
                best_run, best_epoch, best_val = run_num, best_epoch_in_run, best_val_in_run
        except Exception:
            print("Warning: Was not able to compute values for run %s" % run_num)
            continue
    return best_run, best_epoch, best_val, all_vals

In [5]:
def print_validation_profile_and_prior_losses(condition):
    models_path = "/users/amtseng/att_priors/models/trained_models/profile_models/%s/" % condition
    
    print("Best profile loss overall:")
    best_run, best_epoch, best_val, all_vals = get_best_metric_at_best_epoch(
        models_path,
        "val_prof_corr_losses",
        lambda values: np.mean(values),
        lambda x, y: x < y
    )
    print("\tBest run: %s" % best_run)
    print("\tBest epoch in run: %d" % best_epoch)
    print("\tAssociated value: %s" % best_val)
    
    print("Best epoch in each run:")
    for key in sorted(all_vals.keys(), key=lambda p: int(p[0])):
        print("\tRun %s, epoch %d: %6.2f" % (key[0], key[1], all_vals[key]))
        
    print("All validation profile and prior losses:")
    for key in sorted(all_vals.keys(), key=lambda p: int(p[0])):
        print(key[0])
        metrics = import_metrics_json(models_path, key[0])
        print("\t" + " ".join(["%6.2f" % i for i in np.mean(metrics["val_prof_corr_losses"]["values"], axis=1)]))
        print("\t" + " ".join(["%6.4f" % i for i in np.mean(metrics["val_pos_att_losses"]["values"], axis=1)]))

In [8]:
print_validation_profile_and_prior_losses("SPI1")

Best profile loss overall:
	Best run: 33
	Best epoch in run: 10
	Associated value: 88.52770374925001
Best epoch in each run:
	Run 1, epoch 9:  90.26
	Run 2, epoch 9:  89.05
	Run 3, epoch 10:  89.23
	Run 4, epoch 7:  89.27
	Run 5, epoch 10:  88.99
	Run 6, epoch 10:  89.62
	Run 7, epoch 10:  89.21
	Run 8, epoch 10:  89.20
	Run 9, epoch 9:  89.67
	Run 10, epoch 10:  88.76
	Run 11, epoch 10:  89.58
	Run 12, epoch 10:  89.80
	Run 13, epoch 8:  88.60
	Run 14, epoch 10:  88.99
	Run 15, epoch 2:  90.67
	Run 16, epoch 5:  89.56
	Run 17, epoch 10:  89.77
	Run 18, epoch 10:  89.92
	Run 19, epoch 10:  89.03
	Run 20, epoch 10:  89.54
	Run 21, epoch 4:  89.97
	Run 22, epoch 10:  89.68
	Run 23, epoch 10:  89.58
	Run 24, epoch 10:  88.78
	Run 25, epoch 10:  90.40
	Run 26, epoch 10:  88.83
	Run 27, epoch 10:  89.36
	Run 28, epoch 8:  90.63
	Run 29, epoch 10:  89.49
	Run 30, epoch 10:  89.11
	Run 31, epoch 10:  89.05
	Run 32, epoch 8:  89.25
	Run 33, epoch 10:  88.53
	Run 34, epoch 10:  89.71
	Run 35, e

	 91.47  90.94  90.76  90.13  90.36  90.19  90.04  90.16  90.26  90.13
	0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000
49
	 91.32  90.25  89.87  89.66  89.45  89.43  89.19  89.27  89.14  89.26
	0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000
50
	 92.23  90.84  90.05  89.70  89.29  89.16  89.05  88.86  89.00  88.77
	0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000


In [28]:
print_validation_profile_and_prior_losses("SPI1_prior_attinflate")

Best profile loss overall:
	Best run: 2
	Best epoch in run: 5
	Associated value: 89.53327648109575
Best epoch in each run:
	Run 1, epoch 6:  90.34
	Run 2, epoch 5:  89.53
	Run 3, epoch 8:  90.92
	Run 4, epoch 3:  91.03
	Run 5, epoch 7:  90.34
	Run 6, epoch 10:  89.66
	Run 7, epoch 10:  89.81
	Run 8, epoch 9:  90.43
	Run 9, epoch 2:  90.53
All validation profile and prior losses:
1
	 90.92  90.49  90.44  90.47  90.40  90.34  90.46  90.36  90.36  90.51
	0.1634 0.0804 0.0671 0.0627 0.0591 0.0580 0.0552 0.0542 0.0580 0.0562
2
	 90.20  89.73  89.67  89.56  89.53  89.56  89.57  89.54  89.63  89.69
	0.1458 0.0787 0.0676 0.0619 0.0607 0.0562 0.0573 0.0561 0.0552 0.0522
3
	 91.77  91.07  91.21  91.00  91.02  91.37  91.29  90.92  91.10  91.09
	0.1540 0.0751 0.0644 0.0631 0.0614 0.0570 0.0549 0.0553 0.0562 0.0544
4
	 92.09  91.32  91.03
	0.1504 0.0886 0.0705
5
	 90.67  90.69  90.50  90.53  90.77  90.75  90.34  90.46  90.78  90.50
	0.1587 0.0773 0.0666 0.0616 0.0597 0.0562 0.0574 0.0535 0.0552 0.0

In [30]:
print_validation_profile_and_prior_losses("SPI1_prior_attconstant")

Best profile loss overall:
	Best run: 6
	Best epoch in run: 10
	Associated value: 89.43545644086106
Best epoch in each run:
	Run 1, epoch 10:  89.87
	Run 2, epoch 10:  90.15
	Run 3, epoch 10:  89.92
	Run 4, epoch 10:  90.16
	Run 5, epoch 10:  90.54
	Run 6, epoch 10:  89.44
	Run 7, epoch 10:  90.00
	Run 8, epoch 9:  90.21
	Run 9, epoch 6:  90.08
All validation profile and prior losses:
1
	 92.41  91.31  91.02  90.55  90.37  90.21  90.06  90.01  90.04  89.87
	0.0694 0.0594 0.0527 0.0496 0.0498 0.0459 0.0461 0.0446 0.0436 0.0445
2
	 91.27  90.99  90.68  90.31  90.44  90.26  90.31  90.32  90.32  90.15
	0.0650 0.0602 0.0578 0.0557 0.0539 0.0541 0.0530 0.0538 0.0546 0.0544
3
	 93.55  92.03  91.23  90.96  90.70  90.29  90.24  90.08  89.95  89.92
	0.0779 0.0658 0.0582 0.0539 0.0537 0.0497 0.0475 0.0474 0.0451 0.0451
4
	 99.18  96.59  95.51  94.56  91.49  90.82  91.77  90.56  90.97  90.16
	0.0551 0.0460 0.0451 0.0358 0.0548 0.0486 0.0530 0.0459 0.0471 0.0443
5
	 95.47  93.22  92.59  91.76  91.5

In [31]:
print_validation_profile_and_prior_losses("TEAD4")

Best profile loss overall:
	Best run: 17
	Best epoch in run: 9
	Associated value: 203.30705208353476
Best epoch in each run:
	Run 1, epoch 9: 204.14
	Run 2, epoch 10: 206.80
	Run 3, epoch 10: 208.88
	Run 4, epoch 10: 207.57
	Run 5, epoch 9: 204.24
	Run 6, epoch 10: 204.69
	Run 7, epoch 9: 208.48
	Run 8, epoch 9: 208.00
	Run 9, epoch 10: 213.23
	Run 10, epoch 10: 232.69
	Run 11, epoch 10: 204.16
	Run 12, epoch 10: 207.08
	Run 13, epoch 10: 204.09
	Run 14, epoch 10: 206.05
	Run 15, epoch 10: 209.06
	Run 16, epoch 10: 208.27
	Run 17, epoch 9: 203.31
	Run 18, epoch 10: 221.71
	Run 19, epoch 10: 208.10
	Run 20, epoch 10: 228.12
	Run 21, epoch 10: 204.62
	Run 22, epoch 10: 204.42
	Run 23, epoch 10: 225.76
	Run 24, epoch 10: 214.60
	Run 25, epoch 10: 228.51
	Run 26, epoch 10: 207.09
	Run 27, epoch 10: 210.44
	Run 28, epoch 10: 214.78
	Run 29, epoch 10: 212.19
	Run 30, epoch 10: 206.44
	Run 31, epoch 10: 219.68
	Run 32, epoch 10: 205.44
	Run 33, epoch 10: 211.25
	Run 34, epoch 10: 208.67
	Run 

	221.32 216.00 214.35 213.25 211.89 211.00 210.84 211.62 209.84 209.44
	0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000
50
	219.97 212.38 211.45 209.73 208.26 209.54 207.59 206.62 205.48 208.19
	0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000


In [32]:
print_validation_profile_and_prior_losses("TEAD4_prior_attinflate")

Best profile loss overall:
	Best run: 49
	Best epoch in run: 9
	Associated value: 204.70488308443882
Best epoch in each run:
	Run 1, epoch 10: 210.72
	Run 2, epoch 10: 210.21
	Run 3, epoch 10: 205.92
	Run 4, epoch 10: 233.96
	Run 5, epoch 10: 205.85
	Run 6, epoch 10: 216.86
	Run 7, epoch 10: 205.73
	Run 8, epoch 9: 207.97
	Run 9, epoch 10: 207.70
	Run 10, epoch 9: 204.71
	Run 11, epoch 10: 207.35
	Run 12, epoch 10: 220.66
	Run 13, epoch 8: 204.85
	Run 14, epoch 10: 206.26
	Run 15, epoch 10: 207.71
	Run 16, epoch 10: 207.51
	Run 17, epoch 10: 208.24
	Run 18, epoch 10: 208.90
	Run 19, epoch 10: 217.27
	Run 20, epoch 10: 216.59
	Run 21, epoch 8: 207.04
	Run 22, epoch 10: 210.90
	Run 23, epoch 10: 208.92
	Run 24, epoch 10: 211.28
	Run 25, epoch 10: 209.74
	Run 26, epoch 10: 206.93
	Run 27, epoch 8: 209.89
	Run 28, epoch 10: 209.87
	Run 29, epoch 10: 234.06
	Run 30, epoch 10: 232.68
	Run 31, epoch 10: 211.56
	Run 32, epoch 10: 223.92
	Run 33, epoch 9: 208.02
	Run 34, epoch 10: 209.27
	Run 3